# Building a Database for Crime Reports

In this project, we'll build our own database to store data on crimes that have occurred in Boston. We'll look at managing database roles, creating database schemas and tables with proper datatypes, and loading data from CSV files into database tables.

To start, we'll create a database named ***crimes_db*** containing a table inside called ***boston_crimes*** that will contain properly formatted data from our CSV file. Then, we'll create a table inside a schema named ***crimes***. After that, we'll create our `readonly` and `readwrite` groups with the appropriate privileges. Lastly, we'll create a user for each of these groups.

## Introduction to the Data

The dataset we're using for this project can be found in this repository as `boston.csv`. Here are the first few rows to get a better idea for the data we're working with.

In [1]:
import pandas as pd
import psycopg2
boston = pd.read_csv('boston.csv')
boston.head()

,incident_number,offense_code,description,date,day_of_the_week,lat,long
0,1,619,LARCENY ALL OTHERS,2018-09-02,Sunday,42.357791,-71.139371
1,2,1402,VANDALISM,2018-08-21,Tuesday,42.306821,-71.060300
2,3,3410,TOWED MOTOR VEHICLE,2018-09-03,Monday,42.346589,-71.072429
3,4,3114,INVESTIGATE PROPERTY,2018-09-03,Monday,42.334182,-71.078664
4,5,3114,INVESTIGATE PROPERTY,2018-09-03,Monday,42.275365,-71.090361


`incident_number` identifier of the committed crime

`offense_code` numeric identifier code for the committed crime

`description` description of the crime committed

`date` date crime occurred in yyyy-mm-dd format

`day_of_the_week` corresponding day of the week crime occurred

`lat` latitude location where the crime occurred

`long` longitude location where the crime occurred

## Creating the Crime Database & Schema

The first thing we have to do is create our database for storing the crime data. We'll also need to create a schema for containing the tables.

To create a database on your own computer using Postgres, [this post](https://towardsdatascience.com/python-and-postgresql-how-to-access-a-postgresql-database-like-a-data-scientist-b5a9c5a0ea43) explains how to do just that using `Psycopg2`.

In [2]:
# Creating the 'crime_db' database
conn = psycopg2.connect(host="localhost", port = 5432, database="postgres", user="postgres", password="postgres")
conn.autocommit = True # Set to True in order to create a new database
cur = conn.cursor()
cur.execute("CREATE DATABASE crime_db;")
conn.close()

DuplicateDatabase: database "crime_db" already exists


In [3]:
# Creating the 'crimes' schema
conn = psycopg2.connect(dbname='crime_db', user='postgres')
conn.autocommit = True
cur = conn.cursor()
cur.execute('CREATE SCHEMA crimes;')

DuplicateSchema: schema "crimes" already exists


## Obtaining the Column Names

Now that we have our new database and schema, we are ready to start creating tables. 

The first step will be to read in the column names from our CSV file as well as the first row. This is so that we can have easy access to the column headers and first row data while creating our tables.

In [4]:
import csv
with open('boston.csv') as file:
    read = csv.reader(file)
    headers = next(read)
    row_one = next(read)

## Creating an Auxiliary Function

The next step to create our tables will be to create identify the proper datatypes for each column. We'll create a function to compute the distinct values contained within each column.

In [5]:
def get_col_value(csv_filename, col_index):
    values = set()
    with open(csv_filename, 'r') as file:
        next(file)
        read = csv.reader(file)
        for row in read:
            values.add(row[col_index])
    return values

In [6]:
# Print number of distinct values in each column
for i in range(len(headers)):
    values = get_col_value('boston.csv', i)
    print(headers[i], len(values), sep='\t')

incident_number	298329
offense_code	219
description	239
date	1177
day_of_the_week	7
lat	18177
long	18177


## Finding the Max Length

We will also be able to use this function to compute the max length of our text column `description` so that we can select the best sizes of `VARCHAR` to store our columns. `day_of_the_week` is also a text column, but the longest string there we already know will be "Wednesday".

In [7]:
descriptions = get_col_value('boston.csv', 2)
max_len = 0
for description in descriptions:
    max_len = max(max_len, len(description))
print('Max Length:', max_len)

Max Length: 58


## Creating the Table

Now we'll create a table named `boston_crimes` inside of the `crimes` schema within the `crime_db`.



In [9]:
boston.dtypes

incident_number      int64
offense_code         int64
description         object
date                object
day_of_the_week     object
lat                float64
long               float64
dtype: object

Since the `day_of_the_week` column contains a small range of values, we'll create an enumerated datatype. We'll use a decimal type for `lat` and `long`, `VARCHAR(100)` for `description`. 

In [13]:
# Enumerate day_of_the_week
cur.execute("""
    CREATE TYPE weekday AS ENUM ('Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday');
""")

In [14]:
# Create the table
cur.execute("""
    CREATE TABLE crimes.boston_crimes (
        incident_number INTEGER PRIMARY KEY,
        offense_code INTEGER,
        description VARCHAR(100),
        date DATE,
        day_of_the_week weekday,
        lat decimal,
        long decimal
    );
""")

## Loading the Data

The next step is to load in the boston.csv data into the table we just created.

In [15]:
with open('boston.csv') as f:
    cur.copy_expert('COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;', f)
cur.execute('SELECT * FROM crimes.boston_crimes;')

# Checking that the data was loaded properly
print(len(cur.fetchall()))

298329


## Revoking Public Privileges

We've selected the proper datatypes for storing the data, created the table, and loaded in the CSV data. Now we need to create user groups to have `readonly` and `readwrite` permissions by following the least privilege principle to make sure no privileges can be inherited from the public group and public schema and granted by default.

In [16]:
cur.execute('REVOKE ALL ON SCHEMA public FROM public;')
cur.execute('REVOKE ALL ON DATABASE crime_db FROM public;')

## Creating User Groups

In [17]:
# Read only group
cur.execute('CREATE GROUP readonly NOLOGIN;')
cur.execute('GRANT CONNECT ON DATABASE crime_db TO readonly;')
cur.execute('GRANT USAGE ON SCHEMA crimes TO readonly;')
cur.execute('GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;')

In [19]:
# Read write group
cur.execute('CREATE GROUP readwrite NOLOGIN;')
cur.execute('GRANT CONNECT ON DATABASE crime_db TO readwrite;')
cur.execute('GRANT USAGE ON SCHEMA crimes TO readwrite;')
cur.execute('GRANT SELECT, INSERT, UPDATE, DELETE ON ALL TABLES IN SCHEMA crimes TO readwrite;')

In [20]:
cur.execute("CREATE USER data_analyst WITH PASSWORD 'password1';")
cur.execute('GRANT readonly TO data_analyst;')

In [21]:
cur.execute("CREATE USER data_scientist WITH PASSWORD 'password2';")
cur.execute('GRANT readwrite TO data_scientist;')